# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_df = pd.read_csv("output_data/city_data.csv")

city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Humidity,Cloudiness,Max Temperature,Wind Speed,Date
0,0,Barrow,US,71.2906,-156.7887,87,90,39.22,10.36,1628289904
1,1,Vao,NC,-22.6667,167.4833,60,69,67.77,12.17,1628290402
2,2,Ahipara,NZ,-35.1667,173.1667,77,72,57.94,17.29,1628290007
3,3,Atuona,PF,-9.8000,-139.0333,77,17,78.31,18.25,1628289865
4,4,New Norfolk,AU,-42.7826,147.0587,65,100,52.92,4.00,1628289913


In [25]:
del city_data_df["Unnamed: 0"]

In [26]:
city_data_df.head()

,City,Country,Lat,Lng,Humidity,Cloudiness,Max Temperature,Wind Speed,Date
0,Barrow,US,71.2906,-156.7887,87,90,39.22,10.36,1628289904
1,Vao,NC,-22.6667,167.4833,60,69,67.77,12.17,1628290402
2,Ahipara,NZ,-35.1667,173.1667,77,72,57.94,17.29,1628290007
3,Atuona,PF,-9.8000,-139.0333,77,17,78.31,18.25,1628289865
4,New Norfolk,AU,-42.7826,147.0587,65,100,52.92,4.00,1628289913


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
# configure gmaps

gmaps.configure(api_key = g_key)

In [44]:
city_data_df.dtypes

City                object
Country             object
Lat                float64
Lng                float64
Humidity             int64
Cloudiness           int64
Max Temperature    float64
Wind Speed         float64
Date                 int64
dtype: object

In [36]:
#establish locations variable
locations = city_data_df[["Lat", "Lng"]].astype(float)
locations.head()

,Lat,Lng
0,71.2906,-156.7887
1,-22.6667,167.4833
2,-35.1667,173.1667
3,-9.8000,-139.0333
4,-42.7826,147.0587


In [37]:
# establish humidity variable
humidity = city_data_df["Humidity"].astype(float)
humidity.head()

0    87.0
1    60.0
2    77.0
3    77.0
4    65.0
Name: Humidity, dtype: float64

In [45]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
# Remove rows missing data
clean_cities_df = city_data_df.dropna()

In [42]:
# clean_cities_df

In [48]:
# narrow cities to fit weather conditions
narrow_cities_df = clean_cities_df.loc[(clean_cities_df["Max Temperature"] < 80) & (clean_cities_df["Max Temperature"] > 70) \
                                      & (clean_cities_df["Wind Speed"] < 10) \
                                      & (clean_cities_df["Cloudiness"] == 0)]

narrow_cities_df

,City,Country,Lat,Lng,Humidity,Cloudiness,Max Temperature,Wind Speed,Date
55,Dalian,CN,38.9122,121.6022,83,0,76.93,2.24,1628290487
252,Cadiz,ES,36.5000,-5.7500,67,0,72.86,2.64,1628290817
256,Karatau,KZ,43.1667,70.4667,28,0,72.91,1.88,1628290829
283,Shirvan,AZ,39.9509,48.8994,59,0,79.32,4.90,1628290875
300,Krasnyy Yar,RU,46.5331,48.3456,25,0,75.13,9.89,1628290907
482,Neyshabur,IR,36.2133,58.7958,21,0,75.83,3.53,1628291208
491,Arrondissement de Bastia,FR,42.6667,9.3333,74,0,73.29,2.30,1628291222


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
# create hotel_df

hotel_df = narrow_cities_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
55,Dalian,CN,38.9122,121.6022,
252,Cadiz,ES,36.5000,-5.7500,
256,Karatau,KZ,43.1667,70.4667,
283,Shirvan,AZ,39.9509,48.8994,
300,Krasnyy Yar,RU,46.5331,48.3456,
482,Neyshabur,IR,36.2133,58.7958,
491,Arrondissement de Bastia,FR,42.6667,9.3333,


In [59]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Blank field")

hotel_df

,City,Country,Lat,Lng,Hotel Name
55,Dalian,CN,38.9122,121.6022,Kempinski Hotel Dalian
252,Cadiz,ES,36.5000,-5.7500,Hacienda Barriche
256,Karatau,KZ,43.1667,70.4667,Baza Zika
283,Shirvan,AZ,39.9509,48.8994,Nəriman Nərimanov küçəsi Şirvan Azərbaycan
300,Krasnyy Yar,RU,46.5331,48.3456,Cottege near Ufa
482,Neyshabur,IR,36.2133,58.7958,Amiran Hotel
491,Arrondissement de Bastia,FR,42.6667,9.3333,Hotel Santa Maria


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [64]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))